## Методы сбора и обработки данных из сети Интернет
### Урок 4. Парсинг HTML. XPath

In [3]:
#  загружаемые библиотеки
from pprint import pprint
import requests
from lxml import html
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

In [4]:
#  Подключение базы данных MongoDB
client = MongoClient('localhost', 27017)

db = client['news']  # database
yandex = db.yandex  # collection

In [5]:
# #  Очистка коллекции БД:
# yandex.delete_many({})

In [6]:
url = 'https://yandex.ru/news'
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 '
                         'Safari/537.36'}

res = requests.get(url, headers=headers)
dom = html.fromstring(res.text)

topnews = dom.xpath("//a[contains(@href,'rubric=index&fan') and @class='mg-card__link']/ancestor::div[contains("
                    "@class,'mg-grid__item')] ")

In [7]:
for item in topnews:

    source = item.xpath('.//a[@class="mg-card__source-link"]/text()')
    title = item.xpath('.//h2[@class="mg-card__title"]//text()')[0].replace('\xa0', ' ')
    link = item.xpath('.//a[@href]//@href')[0]
    data = item.xpath('.//span[@class="mg-card-source__time"]/text()')

    news_list = {
        'Источник': source,
        'Название': title,
        'Ссылка': link,
        'Дата': data,
    }

    #  Запись документов в БД:
    try:
        yandex.update_one(news_list, {'$set': news_list}, upsert=True)
    except dke:
        print('Duplicate key error collection')

In [8]:
#  Просмотр результата:
for doc in yandex.find({}):
    pprint(doc)
print('Documents in collection: ', db.yandex.count_documents({}))  # Выводим количество документов в БД

{'_id': ObjectId('621d864889dba884ec37e6fa'),
 'Дата': ['05:22'],
 'Источник': ['РИА Новости'],
 'Название': 'МИД Японии объявил о введении санкций против Путина, Лаврова и '
             'Шойгу',
 'Ссылка': 'https://yandex.ru/news/story/MID_YAponii_obyavil_ovvedenii_sankcij_protiv_Putina_Lavrova_i_SHojgu--1fb29052500047ae198f4d05d9998daf?lang=ru&rubric=index&fan=1&stid=AiZujQWg_wtzzgylA3JD&t=1646101684&tt=true&persistent_id=182467823&story=f8dadab8-da5f-5824-bd6c-11af1f669cd4'}
{'_id': ObjectId('621d864889dba884ec37e6fc'),
 'Дата': ['05:21'],
 'Источник': ['РИА Новости'],
 'Название': 'Постпред КНР при ООН Цзюнь: диалог между Москвой и Киевом '
             'поможет решить кризис на Украине',
 'Ссылка': 'https://yandex.ru/news/story/Postpred_KNR_priOON_Czyun_dialog_mezhduMoskvoj_i_Kievom_pomozhet_reshit_krizis_naUkraine--935040a3179955f11d6baa4751684cfe?lang=ru&rubric=index&fan=1&stid=D8daKLoGdxuTvo2pKnG0&t=1646101684&tt=true&persistent_id=182464770&story=a420afa8-5726-5f2d-8102-6f87d